In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 원데이터 가져오기
df = pd.read_excel('/content/drive/MyDrive/유사도 모델/뉴스내용 vs SNS게시물이미지 간 유사도 측정_데이터.xlsx')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1394 entries, 0 to 1393
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   NO             1394 non-null   int64  
 1   플랫폼            1394 non-null   object 
 2   파일명            1394 non-null   object 
 3   게시물이미지         1394 non-null   object 
 4   언론사명           1394 non-null   object 
 5   뉴스주소           1388 non-null   object 
 6   뉴스제목           1380 non-null   object 
 7   뉴스내용           1372 non-null   object 
 8   유사도(코사인)       0 non-null      float64
 9   유사도(자카드_뉴스기준)  0 non-null      float64
dtypes: float64(2), int64(1), object(7)
memory usage: 109.0+ KB


In [ ]:
# 결측치 처리 - NaN를 존재하지 않는 단어로 변경
nan_delete = df.fillna('꺍')

코사인 유사도

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity # cosine_similarity API 사용

rowlen = nan_delete.shape[0] # 행 개수 추출

# 행 개수만큼 반복하며 유사도 측정
for i in range(0, rowlen):
  sentences = (nan_delete['게시물이미지'][i], nan_delete['뉴스내용'][i])
  tfidf_vectorizer = TfidfVectorizer() # td-idf vectorize 사용하여 벡터화 진행
  tfidf_matrix = tfidf_vectorizer.fit_transform(sentences) # 문서 단어 행렬 반환

  cos_similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1]) # cosine_similarity 측정
  df['유사도(코사인)'][i] = cos_similarity[0][0] # cosine_similarity 저장

변형 자카드 유사도 (뉴스 기준)

In [ ]:
rowlen = nan_delete.shape[0]

for i in range(0, rowlen):
  doc1 = nan_delete['게시물이미지'][i]
  doc2 = nan_delete['뉴스내용'][i]

  doc1_tokenized = str(doc1).split()
  doc2_tokenized = str(doc2).split()

  doc_intersection = set(doc1_tokenized).intersection(set(doc2_tokenized))

  # 유사도 측정 - 뉴스 기준
  if len(doc2_tokenized) == 0:
    similarity = 0
  else:
    similarity = len(doc_intersection) / len(doc2_tokenized)


  df['유사도(자카드_뉴스기준)'][i] = similarity

In [ ]:
# 엑셀파일로 데이터프레임 저장

df.to_excel('뉴스내용vsSNS게시물이미지유사도측정_231205.xlsx')